In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [ ]:
from langchain_ollama import OllamaEmbeddings 
from langchain_postgres.vectorstores import PGVector
 
 
# embed each chunk and insert it into the vector store
embeddings_model = OllamaEmbeddings(model="nomic-embed-text",base_url="http://localhost:11434")
# Coba dengan nama default
try:
    db = PGVector.from_existing_index(
        embedding=embeddings_model,
        collection_name="langchain",  # Coba default
        connection=pg_connection,
    )
    print("Berhasil konek dengan collection 'langchain'")
except Exception as e1:
    print(f"Collection 'langchain' tidak ditemukan: {e1}")
    
    # Coba dengan nama lain yang mungkin
    try:
        db = PGVector.from_existing_index(
            embedding=embeddings_model,
            collection_name="nomic-embed-text",  # Nama model
            connection=pg_connection,
        )
        print("Berhasil konek dengan collection 'nomic-embed-text'")
    except Exception as e2:
        print(f"Collection 'nomic-embed-text' tidak ditemukan: {e2}")

Berhasil konek dengan collection 'langchain'


In [3]:
db.similarity_search("Q3", k=4)

[Document(id='1a5729c5-d51d-411e-bb18-4a7db6161250', metadata={'source': '../../tesla.md'}, page_content='Use of Estimates'),
 Document(id='0897e365-990b-48b9-9748-c6e76f0ca622', metadata={'source': '../../tesla.md'}, page_content='---\n\n Warranties'),
 Document(id='48b06a60-2f92-4f82-9eea-40d91831436f', metadata={'source': '../../tesla.md'}, page_content='---\n\n Warranties'),
 Document(id='175eb83c-c4f7-47dd-9311-40f56f7c6be0', metadata={'source': '../../tesla.md'}, page_content='Stock-Based Compensation')]

In [16]:
# create retriever
# retriever = db.as_retriever(search_kwargs={"k": 2})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

retriever = db.as_retriever(search_kwargs={"k": 2})

prompt = ChatPromptTemplate.from_template("""Always answer in indonesian language.Answer the question based only on
    the following context:
{context}

Question: {question}
""")
chain = prompt | llm
rewrite_prompt = ChatPromptTemplate.from_template("""Provide a better search
    query for web search engine to answer the given question, end the queries
    with '**'.
    examples:
    Question: Minyak
    Answer: Jenis Minyak**
    Question: {x} Answer:""")

def parse_rewriter_output(message):
    return message.content.strip('"').strip("**")

rewriter = rewrite_prompt | llm | parse_rewriter_output

 
def qa_rrr(input):
    # rewrite the query
    new_query = rewriter.invoke(input)
    print(f"q:{new_query}")
    # fetch relevant documents
    docs = retriever.get_relevant_documents(new_query)
    # format prompt
    formatted = prompt.invoke({"context": docs, "question": input})
    # generate answer
    answer = llm.invoke(formatted)
    return answer
 
result=qa_rrr("produk tesla?")
print(result.content)

q:produk tesla
Berdasarkan dokumen yang tersedia, produk Tesla meliputi:

1. **Kendaraan listrik berkinerja tinggi** - kendaraan bermotor listrik penuh (fully electric vehicles) yang dirancang, dikembangkan, diproduksi, dijual dan disewakan.

2. **Sistem pembangkit dan penyimpanan energi** - sistem terkait energi yang juga dirancang dan dikembangkan oleh Tesla.

Tesla menjual produk-produknya secara langsung kepada pelanggan dan menekankan pada kinerja, desain yang menarik, serta keselamatan pengguna dan tenaga kerja dalam proses desain dan produksi. Selain itu, Tesla juga terus mengembangkan teknologi mengemudi otomatis penuh untuk meningkatkan keselamatan.


In [8]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Always answer in indonesian language.Answer the question based only on
    the following context:
{context}

Question: {question}
""")
retriever = db.as_retriever(search_kwargs={"k": 2})
perspectives_prompt = ChatPromptTemplate.from_template(
    """You are an AI language model assistant. Your task is to generate five different versions 
    of the given user question to retrieve relevant documents from a vector database.
    By generating multiple perspectives on the user question, your goal is to help the user 
    overcome some of the limitations of the distance-based similarity search. Provide these 
    alternative questions separated by newlines. Original question: {question}"""
)
def parse_queries_output(message):
    return message.content.split('\n')

query_gen = perspectives_prompt | llm | parse_queries_output

def get_unique_union(document_lists):
    # Flatten list of lists, and dedupe them
    deduped_docs = {
        doc.page_content: doc
        for sublist in document_lists for doc in sublist
    }
    # return a flat list of unique docs
    return list(deduped_docs.values())

retrieval_chain = query_gen | retriever.batch | get_unique_union

def multi_query_qa(input):
    # fetch relevant documents
    docs = retrieval_chain.invoke(input)
    # format prompt
    formatted = prompt.invoke({"context": docs, "question": input})
    # generate answer
    answer = llm.invoke(formatted)
    return answer

result=multi_query_qa("produk tesla?")
print(result.content)

Berdasarkan dokumen yang tersedia, produk Tesla mencakup:

1. Kendaraan listrik berperforma tinggi.
2. Sistem pembangkit dan penyimpanan energi.
3. Layanan-layanan yang terkait dengan produk-produk tersebut.

Tesla menjual produknya secara langsung kepada konsumen dan terus mengembangkan infrastruktur layanan pelanggan, termasuk jaringan pusat layanan kendaraan, layanan mobil (Mobile Service), bengkel perbaikan badan kendaraan, serta stasiun pengisian daya Supercharger dan Destination Charger. Perusahaan ini menekankan aspek performa, desain yang menarik, serta keselamatan pengguna dan tenaga kerja dalam proses desain dan produksi. Selain itu, Tesla juga terus mengembangkan teknologi mengemudi otomatis guna meningkatkan keselamatan.


In [2]:

from langchain.prompts import ChatPromptTemplate

#---------------------- START DISINI -------------------------------------
prompt_rag_fusion = ChatPromptTemplate.from_template("""Anda adalah asisten
    yang berguna untuk menghasilkan beberapa kata kunci pencarian berdasarkan
    satu pertanyaan masukan. \n
    Buatlah beberapa kata kunci pencarian yang berkaitan dengan: {question} \n
    Hasil (4 kueri):""")

def parse_queries_output(message):
    return message.content.split('\n')


query_gen = prompt_rag_fusion | llm | parse_queries_output


def reciprocal_rank_fusion(results: list[list], k=60):
    """Reciprocal Rank Fusion (RRF) diterapkan pada beberapa daftar dokumen 
       yang telah diperingkat, dengan parameter opsional k yang digunakan 
       dalam rumus RRF.
    """

    # Inisialisasi dictionary untuk menyimpan skor gabungan setiap dokumen
    # Dokumen akan diindeks berdasarkan isinya untuk memastikan keunikan
    fused_scores = {}
    documents = {}

    # Iterasi melalui setiap daftar dokumen yang telah diperingkat
    for docs in results:
        # Iterasi melalui setiap dokumen dalam daftar,
        # beserta peringkatnya (posisi dalam daftar)
        for rank, doc in enumerate(docs):
            # Gunakan isi dokumen sebagai kunci untuk memastikan keunikan
            doc_str = doc.page_content
            # Jika dokumen belum pernah ditemukan sebelumnya,
            # - inisialisasi skor ke 0
            # - simpan untuk digunakan nanti
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
                documents[doc_str] = doc
            # Perbarui skor dokumen menggunakan rumus RRF:
            # 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Urutkan dokumen berdasarkan skor gabungan secara menurun
    # untuk mendapatkan hasil peringkat ulang akhir
    reranked_doc_strs = sorted(
        fused_scores, key=lambda d : fused_scores[d], reverse=True
    )
    # Ambil dokumen yang sesuai untuk setiap doc_str
    return [
        documents[doc_str]
        for doc_str in reranked_doc_strs
    ]

retrieval_chain = query_gen | retriever.batch | reciprocal_rank_fusion

# Pemakaian
docs=retrieval_chain.invoke({"question":"cabe hujau?"})
print(docs)

NameError: name 'retriever' is not defined

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

prompt_hyde = ChatPromptTemplate.from_template("""Tolong tulis sebuah paragraf untuk
   menjawab pertanyaan.\n Pertanyaan: {question} \n Paragraf:""")

generate_doc = (
    prompt_hyde | llm | StrOutputParser()
)
# generate_doc.invoke({"question":"Siapa elon?"})
retrieval_chain = generate_doc | retriever
# retrieval_chain.invoke({"question":"Siapa elon?"})

def qa(input):
  # ambil dokumen relevan dari rantai pengambilan hyde yang didefinisikan sebelumnya
  docs = retrieval_chain.invoke(input)
  # format prompt
  formatted = prompt.invoke({"context": docs, "question": input})
  # buat jawaban
  chain = llm|StrOutputParser()
  answer = chain.invoke(formatted)
  return answer

qa(" tesla?")

'Tesla, Inc. adalah perusahaan yang didirikan di Negara Bagian Delaware pada tanggal 1 Juli 2003. Perusahaan ini bergerak dalam desain, pengembangan, manufaktur, penjualan, dan pemesanan kendaraan listrik berkinerja tinggi serta sistem pembangkit dan penyimpanan energi. Selain itu, Tesla juga menawarkan layanan-layanan terkait produk-produknya. Perusahaan dipimpin oleh Chief Executive Officer yang bertindak sebagai chief operating decision maker (CODM), dan perusahaan dikelola dalam dua segmen operasional utama, yaitu otomotif dan energi pembangkit serta penyimpanan.'

In [ ]:

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

fields = [
    # AttributeInfo(
    #     name="genre",
    #     description="Genre film",
    #     type="string or list[string]",
    # ),
    # AttributeInfo(
    #     name="year",
    #     description="Tahun film dirilis",
    #     type="integer",
    # ),
    # AttributeInfo(
    #     name="director",
    #     description="Nama sutradara film",
    #     type="string",
    # ),
    # AttributeInfo(
    #     name="rating", description="Penilaian film 1-10", type="float"
    # ),
    AttributeInfo(
        name="source",
        description="Nama source file",
        type="string",
    ),
]
description = "Ringkasan laporan"


retriever = SelfQueryRetriever.from_llm(
    llm, db, description, fields,
)

print(retriever.invoke("Cabe hijau?"))

[Document(id='8817e773-4adb-4b19-841d-dfeff0852c3a', metadata={'source': '../../test.txt'}, page_content='Pemanenan Cabai: Teknik dan Penanganan Pasca Panen\nIndikator dan Teknik Panen\nCabai dapat dipanen pada beberapa tahap kematangan tergantung tujuan pasar:\n\nCabai hijau - dipanen 70-80 hari setelah tanam, sebelum perubahan warna\n\nCabai merah segar - dipanen 85-100 hari setelah tanam, ketika 80-90% permukaan sudah merah\n\nCabai untuk bibit - dipanen pada kematangan fisiologis penuh\n\nPemanenan sebaiknya dilakukan pagi hari setelah embun kering atau sore hari untuk mengurangi stres tanaman. Gunakan gunting atau pisau tajam untuk memotong tangkai buah, meninggalkan sedikit tangkai pada buah untuk memperpanjang masa simpan.\n\nPenanganan Pasca Panen\nPenanganan pasca panen yang tepat menentukan kualitas akhir cabai:\n\nSortasi - memisahkan buah berdasarkan ukuran, warna, dan kondisi\n\nPencucian - membersihkan kotoran dengan air mengalir, diikuti pengeringan\n\nPengemasan - mengg

In [3]:
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities import SQLDatabase
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain.prompts import ChatPromptTemplate

# ganti ini dengan detail koneksi db Anda
db = SQLDatabase.from_uri("sqlite:////mnt/data/userdata/books/learning-langchain/examples/ch02/Chinook.db")
prompt_sql = ChatPromptTemplate.from_template("""
Sebutkan kueri SQL untuk menjawab pertanyaan berikut.
Gunakan dialek: {dialect}
Tampilkan maksimal {top_k} hasil saja.

PENTING: Berikan HANYA kueri SQL saja, tanpa tanda backtick, tanpa tulisan "sql", dan tanpa penjelasan apapun. 
Langsung mulai dengan SELECT.

Berikut adalah struktur tabelnya:
{table_info}

Pertanyaan: {input}
Kueri SQL:""")

# Sekarang panggil chain-nya
write_query = create_sql_query_chain(llm, db, prompt=prompt_sql)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

# LangChain akan otomatis mengisi {dialect}, {table_info}, dan {top_k}
question = "Tampilkan daftar produk"
hasil = chain.invoke({"input": question,"question": question})
print(hasil)


[('For Those About To Rock (We Salute You)',), ('Balls to the Wall',), ('Fast As a Shark',), ('Restless and Wild',), ('Princess of the Dawn',)]
